# 1 Load libraries and data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data= np.load('Preprocessed.npz')
X= data['X']
y= data['y']
test= data['test']

subm= pd.read_csv('subm.csv', index_col= 'PassengerId')

In [3]:
X.shape

(1098, 30)

In [4]:
test.shape

(418, 30)

# 2 XGBoost

In [5]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [6]:
X_train, X_test, y_train, y_test= train_test_split(X,y,train_size=0.9,random_state=42, stratify= y)
X_train, X_valid, y_train, y_valid= train_test_split(X_train,y_train,train_size=0.88875,random_state=42, stratify= y_train)

In [7]:
dtrain = xgb.DMatrix(X_train, label= y_train)
dvalid = xgb.DMatrix(X_valid, label= y_valid)
dtest= xgb.DMatrix(X_test, y_test)

In [8]:
def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "error",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 3, 15)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-error")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation"), (dtrain, 'train')], callbacks=[pruning_callback])
    accuracy = accuracy_score(y_test , np.rint(bst.predict(dtest)))
    return accuracy

In [9]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
    direction="maximize"
)
study.optimize(objective, n_trials=1000)

[0]	validation-error:0.20000	train-error:0.18451
[1]	validation-error:0.20000	train-error:0.17198
[2]	validation-error:0.19091	train-error:0.16515
[3]	validation-error:0.18182	train-error:0.16173
[4]	validation-error:0.18182	train-error:0.16059
[5]	validation-error:0.17273	train-error:0.15604
[6]	validation-error:0.16364	train-error:0.15262
[7]	validation-error:0.16364	train-error:0.15262
[8]	validation-error:0.16364	train-error:0.15376
[9]	validation-error:0.16364	train-error:0.15262
[0]	validation-error:0.20000	train-error:0.14123
[1]	validation-error:0.20000	train-error:0.14123
[2]	validation-error:0.20000	train-error:0.14123
[3]	validation-error:0.20000	train-error:0.14123
[4]	validation-error:0.20000	train-error:0.14123
[5]	validation-error:0.20000	train-error:0.14123
[6]	validation-error:0.20000	train-error:0.14123
[7]	validation-error:0.20000	train-error:0.14123
[8]	validation-error:0.20000	train-error:0.14123
[9]	validation-error:0.20000	train-error:0.14123
[0]	validation-error

In [10]:
print(study.best_value)
xgb_best_params = {**study.best_params, **{
        "verbosity": 1,
        "objective": "binary:logistic",
        "eval_metric": "error",}}
xgb_best_params

0.8545454545454545


{'booster': 'gblinear',
 'lambda': 0.00849025134316801,
 'alpha': 3.9169040444590626e-05,
 'verbosity': 1,
 'objective': 'binary:logistic',
 'eval_metric': 'error'}

In [39]:
xgb_best_params= {
 'booster': 'gblinear',
 'lambda': 0.00849025134316801,
 'alpha': 3.9169040444590626e-05,
 'verbosity': 1,
 'objective': 'binary:logistic',
 'eval_metric': 'error'
}

'booster': 'dart',
 'lambda': 0.6135543718920636,
 'alpha': 0.016994354680641355,
 'max_depth': 7,
 'eta': 1.4353685854998643e-08,
 'gamma': 6.163955967895567e-07,
 'grow_policy': 'lossguide',
 'sample_type': 'uniform',
 'normalize_type': 'tree',
 'rate_drop': 7.104745889921968e-06,
 'skip_drop': 0.0008860153915581865,
 'verbosity': 1,
 'objective': 'binary:logistic',
 'eval_metric': 'error

In [40]:
dbigtrain= xgb.DMatrix(X, label= y)

In [41]:
xgb_model = xgb.train(xgb_best_params, dbigtrain, evals=[(dvalid, "validation"),(dbigtrain, 'train')] )

[0]	validation-error:0.18182	train-error:0.17760
[1]	validation-error:0.18182	train-error:0.16393
[2]	validation-error:0.18182	train-error:0.16029
[3]	validation-error:0.17273	train-error:0.15665
[4]	validation-error:0.16364	train-error:0.15301
[5]	validation-error:0.16364	train-error:0.15210
[6]	validation-error:0.15454	train-error:0.14936
[7]	validation-error:0.15454	train-error:0.14845
[8]	validation-error:0.15454	train-error:0.14572
[9]	validation-error:0.15454	train-error:0.14481


In [42]:
xgb_preds= xgb_model.predict(xgb.DMatrix(test))

In [43]:
subm['Survived']= np.rint(xgb_preds).astype(int)
subm.to_csv('subm.csv')
subm.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


# 3 LightGBM

In [16]:
import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import KFold

In [17]:
lgbm_train = lgb.Dataset(X, label=y)

params = {
    "objective": "binary",
    "metric": "binary_error",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

tuner = lgb.LightGBMTunerCV(
    params,
    lgbm_train,
    num_boost_round= 2000,
    return_cvbooster= True,
    folds=KFold(n_splits=5),
    callbacks=[early_stopping(100), log_evaluation(100)],
)

tuner.run()

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]C:\DS\Anaconda\envs\Env1\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	cv_agg's binary_error: 0.433873 + 0.270618
Training until validation scores don't improve for 100 rounds
[2]	cv_agg's binary_error: 0.331756 + 0.208493
[3]	cv_agg's binary_error: 0.25604 + 0.11182
[4]	cv_agg's binary_error: 0.225936 + 0.0676496
[5]	cv_agg's binary_error: 0.188526 + 0.0199268
[6]	cv_agg's binary_error: 0.176667 + 0.0142896
[7]	cv_agg's binary_error: 0.166638 + 0.0166462
[8]	cv_agg's binary_error: 0.167538 + 0.0226447
[9]	cv_agg's binary_error: 0.165699 + 0.0282936
[10]	cv_agg's binary_error: 0.168435 + 0.0252625
[11]	cv_agg's binary_error: 0.167522 + 0.0264338
[12]	cv_agg's binary_error: 0.166613 + 0.0256813
[13]	cv_agg's binary_error: 0.162968 + 0.0272135
[14]	cv_agg's binary_error: 0.163877 + 0.0293394
[15]	cv_agg's binary_error: 0.159332 + 0.0252952
[16]	cv_agg's binary_error: 0.158418 + 0.0266245
[17]	cv_agg's binary_error: 0.158418 + 0.0263123
[18]	cv_agg's binary_error: 0.1566 + 0.0245752
[19]	cv_agg's binary_error: 0.162051 + 0.0294124
[20]	cv_agg's binary_er

feature_fraction, val_score: 0.151083:  14%|#4        | 1/7 [00:01<00:05,  1.00it/s]

[227]	cv_agg's binary_error: 0.156546 + 0.0550755
[228]	cv_agg's binary_error: 0.156546 + 0.0550755
[229]	cv_agg's binary_error: 0.156546 + 0.0550755
[230]	cv_agg's binary_error: 0.156546 + 0.0550755
[231]	cv_agg's binary_error: 0.15746 + 0.0549327
[232]	cv_agg's binary_error: 0.15746 + 0.0549327
[233]	cv_agg's binary_error: 0.156546 + 0.0566432
[234]	cv_agg's binary_error: 0.156546 + 0.0566432
Early stopping, best iteration is:
[134]	cv_agg's binary_error: 0.151083 + 0.0523661
[1]	cv_agg's binary_error: 0.43296 + 0.271087
[2]	cv_agg's binary_error: 0.314504 + 0.224322
[3]	cv_agg's binary_error: 0.237808 + 0.0998847
[4]	cv_agg's binary_error: 0.202229 + 0.047313
[5]	cv_agg's binary_error: 0.185812 + 0.0238341
[6]	cv_agg's binary_error: 0.185795 + 0.0215023
[7]	cv_agg's binary_error: 0.170282 + 0.0156321
[8]	cv_agg's binary_error: 0.167551 + 0.0180591
[9]	cv_agg's binary_error: 0.16572 + 0.0216038
[10]	cv_agg's binary_error: 0.16572 + 0.0229036
[11]	cv_agg's binary_error: 0.164815 + 0.0

C:\DS\Anaconda\envs\Env1\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[146]	cv_agg's binary_error: 0.172038 + 0.0520397
[147]	cv_agg's binary_error: 0.173856 + 0.0530543
[148]	cv_agg's binary_error: 0.171129 + 0.0519004
[149]	cv_agg's binary_error: 0.171129 + 0.0519799
[150]	cv_agg's binary_error: 0.169315 + 0.0502063
[151]	cv_agg's binary_error: 0.170216 + 0.0536294
[152]	cv_agg's binary_error: 0.171133 + 0.0533438
[153]	cv_agg's binary_error: 0.173865 + 0.0529254
[154]	cv_agg's binary_error: 0.171133 + 0.0515145
[155]	cv_agg's binary_error: 0.171129 + 0.0536389
[156]	cv_agg's binary_error: 0.170216 + 0.0559087
[157]	cv_agg's binary_error: 0.169307 + 0.0556009
Early stopping, best iteration is:
[57]	cv_agg's binary_error: 0.145645 + 0.0377514
[1]	cv_agg's binary_error: 0.46484 + 0.264123
[2]	cv_agg's binary_error: 0.361071 + 0.294963
[3]	cv_agg's binary_error: 0.251453 + 0.064719
[4]	cv_agg's binary_error: 0.205853 + 0.0352212
[5]	cv_agg's binary_error: 0.205849 + 0.0323734
[6]	cv_agg's binary_error: 0.194886 + 0.0240104
[7]	cv_agg's binary_error: 0.185

C:\DS\Anaconda\envs\Env1\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[49]	cv_agg's binary_error: 0.16479 + 0.0309615
[50]	cv_agg's binary_error: 0.163873 + 0.0319264
[51]	cv_agg's binary_error: 0.162964 + 0.033016
[52]	cv_agg's binary_error: 0.160237 + 0.0311027
[53]	cv_agg's binary_error: 0.159328 + 0.0306547
[54]	cv_agg's binary_error: 0.159332 + 0.0299959
[55]	cv_agg's binary_error: 0.159323 + 0.0329874
[56]	cv_agg's binary_error: 0.159328 + 0.0325034
[57]	cv_agg's binary_error: 0.159328 + 0.0343751
[58]	cv_agg's binary_error: 0.160232 + 0.0354609
[59]	cv_agg's binary_error: 0.162051 + 0.0362786
[60]	cv_agg's binary_error: 0.162055 + 0.0339271
[61]	cv_agg's binary_error: 0.162055 + 0.0362981
[62]	cv_agg's binary_error: 0.159323 + 0.036811
[63]	cv_agg's binary_error: 0.157505 + 0.0376848
[64]	cv_agg's binary_error: 0.161146 + 0.0373326
[65]	cv_agg's binary_error: 0.161146 + 0.0374431
[66]	cv_agg's binary_error: 0.160232 + 0.039258
[67]	cv_agg's binary_error: 0.158418 + 0.0362038
[68]	cv_agg's binary_error: 0.158414 + 0.0360684
[69]	cv_agg's binary_err

min_data_in_leaf, val_score: 0.145645: 100%|##########| 5/5 [00:03<00:00,  1.62it/s]

[151]	cv_agg's binary_error: 0.16203 + 0.0485548
[152]	cv_agg's binary_error: 0.162026 + 0.0493012
[153]	cv_agg's binary_error: 0.162935 + 0.0497811
[154]	cv_agg's binary_error: 0.162939 + 0.0489576
[155]	cv_agg's binary_error: 0.163852 + 0.0490186
[156]	cv_agg's binary_error: 0.162947 + 0.0460118
[157]	cv_agg's binary_error: 0.162947 + 0.0463696
Early stopping, best iteration is:
[57]	cv_agg's binary_error: 0.145645 + 0.0377514


In [18]:
print("Best score:", tuner.best_score)
lgbm_best_params = tuner.best_params
lgbm_best_params

Best score: 0.14564549605645496


{'objective': 'binary',
 'metric': 'binary_error',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 31,
 'feature_fraction': 1.0,
 'bagging_fraction': 0.642156294633355,
 'bagging_freq': 6,
 'min_child_samples': 20}

In [44]:
# dpredict = lgb.Dataset(test)
lgbmmodel= tuner.get_best_booster()
lgbm_preds= np.array(lgbmmodel.predict(test))
lgbm_preds = lgbm_preds.mean(axis=0)

In [45]:
subm['Survived']= np.rint(lgbm_preds).astype(int)
subm.to_csv('subm.csv')
subm.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,1
896,1


# 4 CatBoost

In [21]:
# X_train, X_test, y_train, y_test= train_test_split(X,y,train_size=0.8,random_state=42)

In [22]:
import catboost
import optuna
from sklearn.metrics import accuracy_score

In [50]:
def objective1(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "8gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    ctbs = catboost.CatBoostClassifier(**param)

    ctbs.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose= 0, early_stopping_rounds=100)

    preds = ctbs.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

In [51]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
study1 = optuna.create_study(direction= "maximize")
study1.optimize(objective1, n_trials= 1000)

In [52]:
trial1 = study1.best_trial
print("Value: {}".format(trial1.value))
ctbst_best_params= trial1.params
ctbst_best_params

Value: 0.8863636363636364


{'objective': 'Logloss',
 'colsample_bylevel': 0.04721022825475476,
 'depth': 8,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS'}

 'objective': 'Logloss',
 'colsample_bylevel': 0.07204025828261643,
 'depth': 3,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS'

In [53]:
ctbs = catboost.CatBoostClassifier(**ctbst_best_params)
ctbs.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose= 0, early_stopping_rounds=100)

ctbs_preds = ctbs.predict_proba(test)[:,1]
ctbs_preds

array([0.1456709 , 0.399254  , 0.14477808, 0.20398565, 0.5405312 ,
       0.14868334, 0.79057316, 0.09915928, 0.91300657, 0.09255934,
       0.12126177, 0.11140704, 0.99060192, 0.05828796, 0.97076883,
       0.96498962, 0.13895014, 0.36734051, 0.38152327, 0.76209014,
       0.18701882, 0.51747211, 0.93387779, 0.41426032, 0.95613899,
       0.05441966, 0.99283275, 0.29574693, 0.30691882, 0.10138296,
       0.05571608, 0.10741279, 0.65546989, 0.55672945, 0.59159663,
       0.35001912, 0.39018824, 0.53627708, 0.19976387, 0.31283756,
       0.15808112, 0.37683837, 0.09637706, 0.93382568, 0.97618895,
       0.21123673, 0.32244914, 0.11724988, 0.99060149, 0.69339236,
       0.4501615 , 0.1348003 , 0.96256676, 0.92819449, 0.22526623,
       0.07693219, 0.19261947, 0.13757891, 0.06705758, 0.98862815,
       0.20094628, 0.18451816, 0.20162986, 0.67824673, 0.85039652,
       0.94029077, 0.77924051, 0.14739182, 0.72203613, 0.80726425,
       0.76796376, 0.02120268, 0.5447575 , 0.58955159, 0.98878

In [27]:
subm['Survived']= np.rint(ctbs_preds).astype(int)
subm.to_csv('subm.csv')
subm.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


# 5 Keras

In [28]:
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
X_train, X_test, y_train, y_test= train_test_split(X,y,train_size=0.8,random_state=42)

In [30]:
    # # units1= trial.suggest_int('units1', 8 , 256),
    # dropout1= trial.suggest_float('dropout1', 0 , 0.3),

    # # units2= trial.suggest_int('units2',8,256),
    # dropout2= trial.suggest_float('dropout2', 0 , 0.3),

    # # units3= trial.suggest_int('units3',8,256),
    # dropout3= trial.suggest_float('dropout3', 0 , 0.3),

In [31]:
input_shape = [X_train.shape[1]]

def objective(trial):
    model = keras.Sequential([
        layers.Dense(
            units= trial.suggest_int('units1', 8 , 256),
            activation='relu',
            input_shape= input_shape),
        layers.Dropout(trial.suggest_float('dropout1', 0 , 0.3)),

        layers.Dense(
            units= trial.suggest_int('units2', 8 , 256),
            activation='relu'),
        layers.Dropout(trial.suggest_float('dropout2', 0 , 0.3)),

        layers.Dense(
            units= trial.suggest_int('units3', 8 , 256), 
            activation='relu'), 
        layers.Dropout(trial.suggest_float('dropout3', 0 , 0.3)),
        
        layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(
            loss='binary_crossentropy',
            optimizer= "adam",
            metrics='accuracy',
    )

    early_stopping = keras.callbacks.EarlyStopping(
        patience=10,
        min_delta=0.00001,
        restore_best_weights=True,
    )

    model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs= 60,
        callbacks=early_stopping,
        verbose=0,
    )

    preds = np.rint(model.predict(X_valid))
    accuracy = accuracy_score(y_valid, preds)
    return accuracy

In [32]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500)

In [33]:
study.best_value

0.9636363636363636

In [34]:
study.best_params

{'units1': 93,
 'dropout1': 0.15947683449338385,
 'units2': 227,
 'dropout2': 0.194184592763345,
 'units3': 160,
 'dropout3': 0.2492173539726238}

In [48]:
keras_model = keras.Sequential([
    layers.Dense(
        units= 93,
        activation='relu',
        input_shape= input_shape),
    layers.Dropout(0.15947683449338385),

    layers.Dense(
        units= 227,
        activation='relu'),
    layers.Dropout(0.194184592763345),

    layers.Dense(
        units= 160, 
        activation='relu'), 
    layers.Dropout(0.2492173539726238),
    
    layers.Dense(1, activation='sigmoid'),
])

keras_model.compile(
        loss='binary_crossentropy',
        optimizer= "adam",
        metrics='binary_accuracy',
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.00001,
    restore_best_weights=True,
)

keras_model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=100,
    epochs=100,
    callbacks=early_stopping,
    verbose=0,
)

In [49]:
keras_preds= keras_model.predict(test).reshape(418)
subm['Survived']= np.rint(keras_preds).astype(int)
subm.to_csv('subm.csv')
subm.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


# 6 Ensemble

In [117]:
rounder= (lambda x: np.rint(x*2)/2)

In [118]:
xgb_f= np.array(pd.read_csv('xgb.csv')['Survived'])
lgbm_f= rounder(lgbm_preds)
ctbs_f= rounder(ctbs_preds)
keras_f= rounder(keras_preds)

In [125]:
preds= (ctbs_f+ xgb_f+ lgbm_f+ keras_f)/4
preds

array([0.   , 0.25 , 0.   , 0.125, 0.625, 0.   , 0.5  , 0.   , 0.875,
       0.   , 0.   , 0.   , 1.   , 0.   , 1.   , 1.   , 0.   , 0.125,
       0.375, 0.625, 0.   , 0.25 , 1.   , 0.375, 1.   , 0.   , 1.   ,
       0.125, 0.5  , 0.125, 0.   , 0.   , 0.75 , 0.375, 0.375, 0.   ,
       0.25 , 0.5  , 0.   , 0.125, 0.   , 0.5  , 0.   , 1.   , 1.   ,
       0.   , 0.25 , 0.   , 1.   , 0.75 , 0.375, 0.   , 0.875, 1.   ,
       0.25 , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.125, 0.   ,
       0.75 , 0.875, 1.   , 0.875, 0.   , 0.625, 1.   , 0.75 , 0.   ,
       0.375, 0.5  , 1.   , 0.375, 0.125, 1.   , 0.   , 0.75 , 0.75 ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.875, 0.625, 0.75 , 0.875,
       0.375, 0.125, 1.   , 0.   , 0.25 , 0.   , 1.   , 0.   , 0.625,
       0.   , 1.   , 0.   , 0.   , 0.   , 0.875, 0.   , 0.   , 0.   ,
       0.   , 0.125, 0.   , 0.875, 1.   , 0.625, 1.   , 0.   , 0.   ,
       1.   , 0.375, 1.   , 1.   , 0.125, 1.   , 0.   , 0.   , 0.375,
       0.   , 1.   ,

In [120]:
pred_labels= np.rint(preds).astype(int)

In [121]:
subm['Survived']= pred_labels
subm.to_csv('subm.csv')
subm.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
